# Разведочный анализ данных по мошенничеству в транзакциях

В этом ноутбуке проводится разведочный анализ данных о транзакциях для выявления потенциальных паттернов мошенничества и формулирования гипотез, которые могут быть полезны для системы фрод-мониторинга.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

pd.set_option('display.max_columns', None)

: 

In [ ]:
transactions_data = pd.read_parquet('transaction_fraud_data.parquet')
exchange_rates_data = pd.read_parquet('historical_currency_exchange.parquet')

## Первичный осмотр данных

Посмотрим на структуру и основные характеристики загруженных данных.

In [ ]:
print("Пример данных о транзакциях:")
display(transactions_data.head())
print("\nИнформация о структуре данных транзакций:")
transactions_data.info()
print("\nОписательная статистика по транзакциям:")
display(transactions_data.describe())
print("\nПропущенные значения в транзакциях:")
display(transactions_data.isna().sum())

In [ ]:
print("Пример данных о курсах валют:")
display(exchange_rates_data.head())
print("\nИнформация о структуре данных курсов валют:")
exchange_rates_data.info()
print("\nОписательная статистика по курсам валют:")
display(exchange_rates_data.describe())
print("\nПропущенные значения в курсах валют:")
display(exchange_rates_data.isna().sum())

## Распределение транзакций по категориям вендоров

Визуализируем распределение легитимных и мошеннических транзакций по категориям вендоров.

In [ ]:
def create_donut_chart(data_series, chart_title):
    plt.figure(figsize=(8, 6))
    colors = sns.color_palette("pastel")[0:len(data_series)]
    
    wedges, texts, autotexts = plt.pie(
        data_series.values,
        labels=data_series.index,
        autopct=lambda pct: f'{pct:.1f}%' if pct > 1 else '',
        startangle=140,
        colors=colors,
        pctdistance=0.85,
        wedgeprops=dict(width=0.3)
    )

    centre_circle = plt.Circle((0,0),0.70,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)
    
    for autotext in autotexts:
        autotext.set_color('black')
        autotext.set_fontsize(9)

    plt.title(chart_title, fontsize=14, pad=20)
    plt.axis('equal')
    plt.tight_layout()
    plt.show()

In [ ]:
legit_vendor_counts = transactions_data['vendor_category'][transactions_data['is_fraud']==False].value_counts()
print("Распределение легитимных транзакций по категориям вендоров:")
display(legit_vendor_counts)
create_donut_chart(legit_vendor_counts, 'Распределение легитимных транзакций по категориям вендоров')

In [ ]:
fraud_vendor_counts = transactions_data['vendor_category'][transactions_data['is_fraud']==True].value_counts()
print("Распределение мошеннических транзакций по категориям вендоров:")
display(fraud_vendor_counts)
create_donut_chart(fraud_vendor_counts, 'Распределение мошеннических транзакций по категориям вендоров')

## Гипотеза 1: Активность клиента и мошенничество

**Гипотеза:** Уровень активности клиента (количество транзакций) связан с вероятностью совершения мошеннической транзакции. Клиенты с очень низкой или очень высокой активностью могут быть более подвержены риску.

**Ценность:** Позволяет сегментировать клиентов по риску на основе их поведения. Это может быть использовано для адаптации уровня контроля и проверок: более активные клиенты могут требовать усиленного мониторинга, а клиенты с аномальной активностью могут быть временно заблокированы или подвергнуты дополнительной аутентификации.

In [ ]:
customer_activity = transactions_data.groupby('customer_id').agg(
    total_transactions=('transaction_id', 'count'),
    fraud_cases=('is_fraud', 'sum')
).reset_index()

customer_activity['is_fraudster'] = customer_activity['fraud_cases'] > 0
customer_activity['activity_level'] = pd.cut(customer_activity['total_transactions'], 
                                             bins=[0, 5, 20, 50, float('inf')], 
                                             labels=['Низкая (1-5)', 'Средняя (6-20)', 'Высокая (21-50)', 'Очень высокая (50+)'])

fraud_by_activity = customer_activity.groupby('activity_level').agg(
    total_customers=('customer_id', 'count'),
    fraudsters=('is_fraudster', 'sum')
).reset_index()

fraud_by_activity['fraud_rate'] = fraud_by_activity['fraudsters'] / fraud_by_activity['total_customers']
display(fraud_by_activity)

In [ ]:
plt.figure(figsize=(6, 4))
sns.barplot(data=fraud_by_activity, x='activity_level', y='fraud_rate', palette='viridis')
plt.title('Доля мошенников по уровню активности клиентов')
plt.xlabel('Уровень активности')
plt.ylabel('Доля мошенников')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.7)
plt.tight_layout()
plt.show()

## Гипотеза 2: Время суток и мошенничество

**Гипотеза:** Мошеннические транзакции чаще происходят в определенные часы суток, возможно, когда клиенты менее бдительны или система мониторинга менее активна.

**Ценность:** Позволяет оптимизировать распределение ресурсов системы фрод-мониторинга. В часы пиковой активности мошенников можно усиливать автоматические проверки и уведомлять операторов о повышенной нагрузке.

In [ ]:
transactions_data['hour'] = pd.to_datetime(transactions_data['timestamp']).dt.hour

fraud_by_hour = transactions_data.groupby('hour').agg(
    total_transactions=('transaction_id', 'count'),
    fraud_cases=('is_fraud', 'sum')
).reset_index()

fraud_by_hour['fraud_rate'] = fraud_by_hour['fraud_cases'] / fraud_by_hour['total_transactions']
display(fraud_by_hour.nlargest(5, 'fraud_rate'))

In [ ]:
plt.figure(figsize=(10, 4))
sns.lineplot(data=fraud_by_hour, x='hour', y='fraud_rate', marker='o', color='red')
plt.title('Доля мошенничества по часам суток')
plt.xlabel('Час суток')
plt.ylabel('Доля мошенничества')
plt.grid(True, alpha=0.7)
plt.xticks(range(0, 24))
plt.tight_layout()
plt.show()

## Гипотеза 3: Интенсивность использования device_fingerprint

**Гипотеза:** Устройства, с которых совершается аномально большое количество транзакций (независимо от количества пользователей), имеют повышенный риск быть скомпрометированными или использоваться ботами.

**Ценность:** Позволяет выявлять устройства с подозрительной активностью. Такие устройства могут быть добавлены в список наблюдения или временно заблокированы для предотвращения дальнейших потенциально мошеннических действий.

In [ ]:
device_analysis = transactions_data.groupby('device_fingerprint').agg(
    unique_customers=('customer_id', 'nunique'),
    total_transactions=('transaction_id', 'count'),
    fraud_cases=('is_fraud', 'sum')
).reset_index()

device_analysis['fraud_ratio'] = device_analysis['fraud_cases'] / device_analysis['total_transactions']
device_analysis['transactions_per_user'] = device_analysis['total_transactions'] / device_analysis['unique_customers']

# Фильтруем устройства с высокой активностью (например, более 50 транзакций)
high_activity_devices = device_analysis[device_analysis['total_transactions'] > 50]
high_activity_devices_sorted = high_activity_devices.sort_values(by='fraud_ratio', ascending=False)
display(high_activity_devices_sorted.head(10))

## Гипотеза 4: Нагрузка на IP-адрес

**Гипотеза:** IP-адреса, с которых инициируется большое количество транзакций, могут указывать на прокси-сервера, ботнеты или просто на общие точки доступа с высокой нагрузкой, требующие дополнительного внимания.

**Ценность:** Помогает идентифицировать IP-адреса с аномальной активностью. Это может быть полезно для создания динамических черных списков или для применения дополнительных правил проверки для транзакций с таких IP.

In [ ]:
ip_analysis = transactions_data.groupby('ip_address').agg(
    unique_customers=('customer_id', 'nunique'),
    total_transactions=('transaction_id', 'count'),
    fraud_cases=('is_fraud', 'sum')
).reset_index()

ip_analysis['fraud_ratio'] = ip_analysis['fraud_cases'] / ip_analysis['total_transactions']
# Фильтруем IP с высокой нагрузкой (например, более 100 транзакций)
high_load_ips = ip_analysis[ip_analysis['total_transactions'] > 100]
high_load_ips_sorted = high_load_ips.sort_values(by='fraud_ratio', ascending=False)
display(high_load_ips_sorted.head(10))

## Гипотеза 5: Категория вендора и мошенничество

**Гипотеза:** Некоторые категории вендоров (мерчантов) имеют статистически более высокий уровень мошенничества по сравнению с другими. Это может быть связано с типом товаров/услуг, политикой безопасности мерчанта или спецификой аудитории.

**Ценность:** Позволяет сегментировать риски по категориям мерчантов. Для категорий с высоким риском можно вводить дополнительные проверки транзакций, более строгие лимиты или даже временно приостанавливать возможность оплаты до ручной проверки.

In [ ]:
fraud_by_vendor = transactions_data.groupby('vendor_category').agg(
    total_transactions=('transaction_id', 'count'),
    fraud_cases=('is_fraud', 'sum')
).reset_index()

fraud_by_vendor['fraud_percentage'] = (fraud_by_vendor['fraud_cases'] / fraud_by_vendor['total_transactions']) * 100
fraud_by_vendor = fraud_by_vendor.sort_values('fraud_percentage', ascending=False)
display(fraud_by_vendor)

In [ ]:
# Хи-квадрат тест
contingency_vendor_tbl = pd.crosstab(transactions_data['vendor_category'], transactions_data['is_fraud'])
chi2_v_stat, p_v_val, dof_v, expected_v = chi2_contingency(contingency_vendor_tbl)

alpha_level = 0.05
print(f"Результаты Хи-квадрат теста для vendor_category и is_fraud:")
print(f"p-value: {p_v_val:.4f}")
if p_v_val > alpha_level:
    print("Нет статистически значимой связи между категорией вендора и мошенничеством.")
else:
    print("Существует статистически значимая связь между категорией вендора и мошенничеством.")

## Гипотеза 6: Географический паттерн мошенничества

**Гипотеза:** Транзакции, совершенные за пределами страны клиента, имеют значительно более высокий уровень мошенничества по сравнению с транзакциями внутри страны.

**Ценность:** Это подтверждение позволяет внедрить усиленную аутентификацию (например, 2FA, SMS-подтверждение) для всех международных транзакций, тем самым снижая риски и повышая безопасность клиентов при поездках или онлайн-покупках за рубежом.

In [ ]:
fraud_by_geo = transactions_data.groupby('is_outside_home_country').agg(
    total=('transaction_id', 'count'),
    fraud_count=('is_fraud', 'sum')
).reset_index()

fraud_by_geo['fraud_percentage'] = (fraud_by_geo['fraud_count'] / fraud_by_geo['total']) * 100
display(fraud_by_geo)

In [ ]:
geo_plot_data = fraud_by_geo.copy()
geo_plot_data['location'] = geo_plot_data['is_outside_home_country'].map({True: 'Вне страны', False: 'Внутри страны'})

plt.figure(figsize=(6, 4))
sns.barplot(data=geo_plot_data, x='location', y='fraud_percentage', palette='viridis')
plt.title('Процент мошеннических транзакций в зависимости от местоположения')
plt.xlabel('Местоположение транзакции')
plt.ylabel('Процент мошенничества (%)')
plt.ylim(0, max(geo_plot_data['fraud_percentage']) * 1.1)
plt.grid(axis='y', alpha=0.7)
plt.tight_layout()
plt.show()

## Heatmap: Домашняя страна клиента vs. Страна транзакции

Визуализация пар "домашняя страна клиента - страна транзакции" для мошеннических операций помогает выявить наиболее подозрительные маршруты. Это может указывать на организованные схемы мошенничества из определенных регионов.

**Ценность:** Позволяет выявлять пары стран с высоким уровнем мошеннической активности. Это может быть использовано для усиления контроля на границе этих стран (например, дополнительная проверка для клиентов из страны A, совершающих покупки в стране B) или даже для временного ограничения таких транзакций при высоком уровне риска.

In [ ]:
# Определяем "домашнюю" страну клиента как самую частую страну его транзакций
home_nation = (
    transactions_data.groupby("customer_id")["country"]
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0])
    .reset_index()
    .rename(columns={"country": "home_country"})
)

# Объединяем с данными о мошенничестве
fraud_transactions = transactions_data[transactions_data["is_fraud"] == True]
fraud_with_home = fraud_transactions.merge(home_nation, on="customer_id", how="left")

# Создаем сводную таблицу
geo_pivot = (
    fraud_with_home.groupby(["home_country", "country"])
    .size()
    .reset_index(name="fraud_incidents")
)

geo_matrix = geo_pivot.pivot(index="home_country", columns="country", values="fraud_incidents").fillna(0)

# Визуализация heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(geo_matrix, annot=False, fmt="g", cmap="YlOrRd", linewidths=0.3, linecolor='gray')
plt.title("Heatmap: Домашняя страна клиента vs. Страна транзакции (Только мошенничество)", fontsize=12)
plt.xlabel("Страна транзакции")
plt.ylabel("Домашняя страна клиента")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()